In [2]:
# forex_plot_utils.ipynb

# IMPORTS
import pandas as pd
import matplotlib.pyplot as plt
import os

# FUNCTION DEFINITION
def plot_forex_csvs(csv1_path, csv2_path, csv3_path, plot_title,
                    historical_color='black', predicted_color='red',
                    actual_future_color='green', vertical_line_color='gray',
                    vertical_line_width=2, output_plot_path=None):
    """
    Loads up to 3 CSV files and plots the historical, predicted, and actual future <CLOSE> price series.
    """

    def load_csv_with_datetime(csv_path):
        df = pd.read_csv(csv_path, sep='\t' if '\t' in open(csv_path).readline() else ',')
        df['DATETIME'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'], format='%Y.%m.%d %H:%M:%S')
        return df[['DATETIME', '<CLOSE>']].reset_index(drop=True)

    def plot_all_series(historical_df, predicted_df, actual_future_df):
        plt.figure(figsize=(14, 6))
        plotted = False

        if historical_df is not None:
            plt.plot(historical_df['DATETIME'], historical_df['<CLOSE>'],
                     label='Historical', color=historical_color)
            plotted = True

        if actual_future_df is not None:
            plt.plot(actual_future_df['DATETIME'], actual_future_df['<CLOSE>'],
                     label='Actual Future', color=actual_future_color)
            plotted = True

        if predicted_df is not None:
            plt.plot(predicted_df['DATETIME'], predicted_df['<CLOSE>'],
                     label='Predicted', color=predicted_color, linestyle='--')
            plotted = True

        if historical_df is not None:
            prediction_start_time = historical_df['DATETIME'].iloc[-1]
            plt.axvline(x=prediction_start_time, color=vertical_line_color,
                        linestyle='--', linewidth=vertical_line_width, label='Prediction Start')

        plt.title(plot_title)
        plt.xlabel('Time')
        plt.ylabel('Close Price')
        if plotted:
            plt.legend()
        plt.grid(True)
        plt.tight_layout()

        if output_plot_path:
            plt.savefig(output_plot_path)
        plt.show()

    # Load datasets
    historical_df = load_csv_with_datetime(csv1_path) if os.path.exists(csv1_path) else None
    predicted_df = load_csv_with_datetime(csv2_path) if os.path.exists(csv2_path) else None
    actual_future_df = load_csv_with_datetime(csv3_path) if os.path.exists(csv3_path) else None

    if historical_df is None:
        print(f"⚠️ Warning: '{csv1_path}' not found. Skipping historical candle plotting.")
    if predicted_df is None:
        print(f"⚠️ Warning: '{csv2_path}' not found. Skipping predicted candle plotting.")
    if actual_future_df is None:
        print(f"⚠️ Warning: '{csv3_path}' not found. Skipping actual future candle plotting.")

    # Plot
    plot_all_series(historical_df, predicted_df, actual_future_df)
